In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [3]:
def get_numpy_data(data, features, output):
    features = ['constant'] + features # this is how you combine two lists
    # create a new col in dataframe named 'constant'
    data['constant'] = 1 
    
    output_array = data[output].to_numpy()

    feature_matrix = data[features].to_numpy()

    return(feature_matrix, output_array)

def predict_output(feature_matrix, weights):
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In [4]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis=0)
    normalize_features = features/norms
    return (normalize_features, norms)

Review of Coordinate Descent

- We seek to obtain a sparse set of weights by minimizing the LASSO cost function
- <span style="color:red">_SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)._</span>
- The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use coordinate descent: at each iteration, we will fix all weights but weight i and find the value of weight i that minimizes the objective. That is, we look for
- <span style="color:gray">argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]</span>
- where all weights other than w[i] are held to be constant. We will optimize one w[i] at a time, circling through the weights multiple times.
    - Pick a coordinate i
    - Compute w[i] that minimizes the LASSO cost function
    - Repeat the two steps for all coordinates, multiple times
- For this assignment, we use cyclical coordinate descent with normalized features, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows: 


       ┌ (ro[i] + lambda/2)    if ro[i] < -lambda/2
w[i] = ├ 0                     if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)    if ro[i] > lambda/2
       
where
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].

Note that we do not regularize the weight of the constant feature (intercept) w[0], so, for this weight, the update is simply:<br>
w[0] = ro[i]

Effect of L1 penalty

- Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.
    - First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire ‘sales’ dataset for now.
    - Normalize columns of the feature matrix. Save the norms of original features as ‘norms’.
    - Set initial weights to [1,4,1].
    - Make predictions with feature matrix and initial weights.
    - Compute values of ro[i], where

ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]

***Quiz Question: Recall that, whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?***

***Quiz Question: What range of values of l1_penalty would set both w[1] and w[2] to zero, if we were to take a step in that coordinate? ***

In [5]:
features = ['sqft_living', 'bedrooms']
output = ['price']
weights = [1.,4.,1.] #intial weight
(feature_matrix, outputArray) = get_numpy_data(sales, features, output)
(feature_matrix, norms) = normalize_features(feature_matrix)
predictions = predict_output(feature_matrix, weights)



In [8]:
ro = [0 for i in range((feature_matrix.shape)[1])]
for j in range((feature_matrix.shape)[1]): 
    ro[j] = sum(feature_matrix[:,j]*(outputArray.flatten()-predictions)+np.dot(weights[j],feature_matrix[:,j]))

In [10]:
ro

[79400446.02812779, 87940004.76666003, 80966839.38462102]

According to formulae: <br>
$-\frac{\lambda}{2} \leq ro[i] \leq \frac{\lambda}{2}$ <br>
$\lambda \geq -2*ro[i]$ and $\lambda \geq 2*ro[i]$ <br> 
For both condition to be satisfied
$\lambda \geq 2*ro[i]$

***Answer to quiz question 1***

In [12]:
print(2*ro[1]) # this will set w[1] to zero
print(2*ro[2]) # this will set w[2] to zero
print("There the answer is between the abve two numbers excluding ",2*ro[1])

175880009.53332007
161933678.76924205
There the answer is between the abve two numbers excluding  175880009.53332007


*** Answer to quiz question 2 ***

In [72]:
print("Value greater the {0} would set both w[1] and w[2] to zero".format(2*ro[2]))

Value greater the 175879202.95546168 would set both w[1] and w[2] to zero


### Single Coordinate Descent Step

In [21]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    
    #ro_i = sum(feature_matrix[:,i]*(output.flatten()-prediction)+np.dot(weights[i],feature_matrix[:,i]))
    ro_i = sum((feature_matrix[:,i] * (output.flatten() - prediction + (weights[i] * feature_matrix[:,i]))))
    
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = (ro_i + l1_penalty/2.)
    elif ro_i > l1_penalty/2.:
        new_weight_i = (ro_i - l1_penalty/2.)
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [22]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


## Cyclical coordinate descent

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat. <br>
When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:

    As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
    After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**

In [23]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    D = feature_matrix.shape[1]
    weights = np.array(initial_weights)
    change = np.array(initial_weights) * 0.0
    converged = False

    while not converged:
        for idx in range(D):
            new_weight = lasso_coordinate_descent_step(idx, feature_matrix,output, weights,l1_penalty)
            # compute change in weight for feature
            change[idx] = np.abs(new_weight - weights[idx])
            weights[idx] = new_weight
        max_change = max(change)
        if max_change < tolerance:
            converged = True
    return weights

In [24]:
#Using the following parameters, learn the weights on the sales dataset.
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [25]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [26]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,initial_weights, l1_penalty, tolerance)

In [27]:
weights

array([21624997.95951872, 63157247.20788978,        0.        ])

In [28]:
prediction =  predict_output(normalized_simple_feature_matrix, weights)
RSS = np.dot(output-prediction, output-prediction)
print('RSS for normalized dataset = ', RSS)

RSS for normalized dataset =  1630492476715384.5


**Quiz Question: What is the RSS of the learned model on the normalized dataset?** <br>
Answer: 1630492476715384.5 <br> <br>
**Which features had weight zero at convergence?**<br>
Answer: Bedroom

## Evaluating LASSO fit with more features

TODO